In [48]:

import numpy as np
import pandas as pd

In [49]:
credits = pd.read_csv('tmdb_5000_credits.csv')
movies = pd.read_csv('tmdb_5000_movies.csv')

In [50]:
#merge both dataframes

movies = movies.merge(credits,on="title")
movies.head(1)

,budget,genres,homepage,id,keywords,original_language,original_title,overview,popularity,production_companies,...,runtime,spoken_languages,status,tagline,title,vote_average,vote_count,movie_id,cast,crew
0,237000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",http://www.avatarmovie.com/,19995,"[{""id"": 1463, ""name"": ""culture clash""}, {""id"":...",en,Avatar,"In the 22nd century, a paraplegic Marine is di...",150.437577,"[{""name"": ""Ingenious Film Partners"", ""id"": 289...",...,162.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}, {""iso...",Released,Enter the World of Pandora.,Avatar,7.2,11800,19995,"[{""cast_id"": 242, ""character"": ""Jake Sully"", ""...","[{""credit_id"": ""52fe48009251416c750aca23"", ""de..."


In [51]:
#columns to  keep  -  id , keywords , title , overview , cast , crew

movies= movies[['movie_id','title','overview','genres','cast','crew','keywords']]
movies.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4809 entries, 0 to 4808
Data columns (total 7 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   movie_id  4809 non-null   int64 
 1   title     4809 non-null   object
 2   overview  4806 non-null   object
 3   genres    4809 non-null   object
 4   cast      4809 non-null   object
 5   crew      4809 non-null   object
 6   keywords  4809 non-null   object
dtypes: int64(1), object(6)
memory usage: 263.1+ KB


In [52]:
movies.isnull().sum()

movie_id    0
title       0
overview    3
genres      0
cast        0
crew        0
keywords    0
dtype: int64

In [53]:
movies.dropna(inplace=True)

In [54]:
movies.iloc[0].genres

'[{"id": 28, "name": "Action"}, {"id": 12, "name": "Adventure"}, {"id": 14, "name": "Fantasy"}, {"id": 878, "name": "Science Fiction"}]'

In [55]:
import ast
def convert(obj):
    l = []
    for i in ast.literal_eval(obj):
        l.append(i['name'])
    return l

def convertcast(obj):
    l = []
    counter=0
    for i in ast.literal_eval(obj):
        if(counter!=3):
            l.append(i['name'])
            counter+=1
    return l

def convertcrew(obj):
    l = []
    for i in ast.literal_eval(obj):
        if(i['job']=='Director'):
            l.append(i['name'])
            break
    return l
  

In [56]:
movies['genres']=movies['genres'].apply(convert)
movies['keywords']=movies['keywords'].apply(convert)
movies['cast']=movies['cast'].apply(convertcast)
movies['crew']=movies['crew'].apply(convertcrew)
movies['overview'] = movies['overview'].apply(lambda x : x.split())

In [57]:
movies['genres'] = movies['genres'].apply(lambda x: [i.replace(" ","") for i in x])
movies['keywords'] = movies['keywords'].apply(lambda x: [i.replace(" ","") for i in x])
movies['cast'] = movies['cast'].apply(lambda x: [i.replace(" ","") for i in x])
movies['crew'] = movies['crew'].apply(lambda x: [i.replace(" ","") for i in x])


In [58]:
movies['tags'] = movies ['overview']+movies['genres']+movies['keywords']+movies['cast']+movies['crew']

In [59]:
new_df = movies[['movie_id','title','tags']]
new_df['tags'] = new_df['tags'].apply(lambda x : " ".join(x))

C:\Users\MAYANK\AppData\Local\Temp\ipykernel_14164\2174986103.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  new_df['tags'] = new_df['tags'].apply(lambda x : " ".join(x))


In [60]:
new_df['tags'] = new_df['tags'].apply(lambda x : x.lower())
new_df

C:\Users\MAYANK\AppData\Local\Temp\ipykernel_14164\3329521041.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  new_df['tags'] = new_df['tags'].apply(lambda x : x.lower())


,movie_id,title,tags
0,19995,Avatar,"in the 22nd century, a paraplegic marine is di..."
1,285,Pirates of the Caribbean: At World's End,"captain barbossa, long believed to be dead, ha..."
2,206647,Spectre,a cryptic message from bond’s past sends him o...
3,49026,The Dark Knight Rises,following the death of district attorney harve...
4,49529,John Carter,"john carter is a war-weary, former military ca..."
...,...,...,...
4804,9367,El Mariachi,el mariachi just wants to play his guitar and ...
4805,72766,Newlyweds,a newlywed couple's honeymoon is upended by th...
4806,231617,"Signed, Sealed, Delivered","""signed, sealed, delivered"" introduces a dedic..."
4807,126186,Shanghai Calling,when ambitious new york attorney sam is sent t...


In [61]:
#applying stemming : 
from nltk.stem.porter import PorterStemmer
ps = PorterStemmer()

In [62]:

def stem(text):
    y = []
    for i  in  text.split():
        y.append(ps.stem(i))
    return " ".join(y)

In [63]:
new_df['tags'] = new_df['tags'].apply(stem)

C:\Users\MAYANK\AppData\Local\Temp\ipykernel_14164\3213734980.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  new_df['tags'] = new_df['tags'].apply(stem)


In [64]:
#converting tags to vectors
from sklearn.feature_extraction.text import CountVectorizer
cv = CountVectorizer(max_features=5000,stop_words='english')

vectors = cv.fit_transform(new_df['tags']).toarray()




In [83]:
#using cosine distance to find similarity between vectors : angle between them
from sklearn.metrics.pairwise import cosine_similarity
similarity = cosine_similarity(vectors)

array([[1.        , 0.08346223, 0.0860309 , ..., 0.04499213, 0.        ,
        0.        ],
       [0.08346223, 1.        , 0.06063391, ..., 0.02378257, 0.        ,
        0.02615329],
       [0.0860309 , 0.06063391, 1.        , ..., 0.02451452, 0.        ,
        0.        ],
       ...,
       [0.04499213, 0.02378257, 0.02451452, ..., 1.        , 0.03962144,
        0.04229549],
       [0.        , 0.        , 0.        , ..., 0.03962144, 1.        ,
        0.08714204],
       [0.        , 0.02615329, 0.        , ..., 0.04229549, 0.08714204,
        1.        ]], shape=(4806, 4806))

In [84]:
def recommend(movie):
    movie_index = new_df[new_df['title']==movie].index[0]
    distances = similarity[movie_index]
    movies_list = sorted(list(enumerate(distances)),reverse=True , key=lambda x :x[1])[1:6]
    for i  in movies_list:
        print(new_df.iloc[i[0]].title)
    return 

recommend('Avatar')

Aliens vs Predator: Requiem
Aliens
Falcon Rising
Independence Day
Titan A.E.


In [67]:
#converting movies df to dic to send in frontend
import pickle
pickle.dump(new_df.to_dict(),open('movies.pkl','wb'))

In [68]:
pickle.dump(similarity,open('similarity.pkl','wb'))
